# Informer Paddle Demo

### 安装依赖包
- `poetry install`
- 或者用pip安装相应包

In [1]:
import paddle
import pandas as pd
import numpy as np
from utils.tools import dotdict
from exp.exp_informer import Exp_Informer

### 打印基本信息

In [2]:
from paddle import utils

utils.run_check()

Running verify PaddlePaddle program ... 
PaddlePaddle works well on 1 CPU.
PaddlePaddle is installed successfully! Let's start deep learning with PaddlePaddle now.


In [3]:
print(f'numpy version: {np.__version__}')
print(f'paddle version: {paddle.__version__}')
print(f'pandas version: {pd.__version__}')
if paddle.is_compiled_with_cuda():
    gpu_count = paddle.device.cuda.device_count()
    print(f'🖥️Use GPU: cuda:{paddle.CUDAPlace(0)},🔢可用GPU数量:{gpu_count}')
else:
    print(f'🔳GPU不可用,将使用{paddle.CPUPlace()}')

numpy version: 1.26.4
paddle version: 3.0.0-beta1
pandas version: 2.2.2
🔳GPU不可用,将使用Place(cpu)


# 训练
### 参数

In [1]:
from informer_paddle.wth_args import get_wth_args_nb

args = get_wth_args_nb()

🔣 args:{'model': 'informer', 'data': 'WTH_small', 'root_path': './data/', 'data_path': 'WTH_small.csv', 'features': 'M', 'target': 'WetBulbCelsius', 'freq': 'h', 'checkpoints': './checkpoints/', 'seq_len': 96, 'label_len': 48, 'pred_len': 24, 'enc_in': 12, 'dec_in': 12, 'c_out': 12, 'd_model': 512, 'n_heads': 8, 'e_layers': 2, 'd_layers': 1, 's_layers': '3,2,1', 'd_ff': 2048, 'factor': 5, 'padding': 0, 'distil': True, 'dropout': 0.05, 'attn': 'prob', 'embed': 'timeF', 'activation': 'gelu', 'output_attention': True, 'mix': True, 'num_workers': 0, 'itr': 1, 'train_epochs': 6, 'batch_size': 32, 'patience': 4, 'learning_rate': 0.0001, 'des': 'test', 'loss': 'mse', 'lradj': 'type1', 'use_amp': False, 'inverse': False, 'use_gpu': False, 'gpu': 0, 'use_multi_gpu': False, 'devices': '0,1,2,3', 'detail_freq': 'h'}


### 训练

In [ ]:
from informer_paddle.args import get_setting

Exp = Exp_Informer
setting = ""
for ii in range(args.itr):
    # setting record of experiments
    setting = get_setting(args, ii)

exp = Exp(args)  # set experiments
print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
exp.train(setting)

print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
exp.test(setting)

if args.do_predict:
    print('>>>>>>>predicting : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
exp.predict(setting, True)

paddle.device.cuda.empty_cache()
print('🚩执行结束🚩')

🏿Use CPU:Place(cpu)
>>>>>>>start training : informer_WTH_small_ftM_sl96_ll48_pl24_dm512_nh8_el2_dl1_df2048_atprob_fc5_ebtimeF_dtTrue_mxTrue_test_0>>>>>>>>>>>>>>>>>>>>>>>>>>
📑_get_data处理结束,flag:【train】,data_set.length:【10605】
📑_get_data处理结束,flag:【val】,data_set.length:【1509】
📑_get_data处理结束,flag:【test】,data_set.length:【3041】
